In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import statsmodels.api as sm
import datetime

In [2]:
df = pd.read_csv('Dataset.csv')
# df.year = pd.to_datetime(df.year,format='%Y-%m-%d')

In [3]:
def timestampToYear(s):
    return datetime.datetime.strptime(s, '%Y-%m-%d').year

In [4]:
df.year = df.year.apply(timestampToYear)

In [5]:
df.set_index('year',inplace=True)

In [6]:
def olsEstimate(series,value,country):
    model = sm.OLS(series, sm.add_constant(series.index)).fit()
    result = pd.DataFrame({'coef':model.params,'p-value':model.pvalues,'country':country,'value':value})
    result.index = ['constant','trend']
    result = result.reset_index()
    return result

In [7]:
LOGRLP = olsEstimate(df.LOGRLP,'LOGRLP','Japan')
LOGRM = olsEstimate(df.LOGRM,'LOGRM','Japan')
LOGRX = olsEstimate(df.LOGRX,'LOGRX','Japan')
dif_LOGRLP = olsEstimate(df.dif_LOGRLP,'dif_LOGRLP','Japan')
dif_LOGRM = olsEstimate(df.dif_LOGRM,'dif_LOGRM','Japan')
dif_LOGRX = olsEstimate(df.dif_LOGRX,'dif_LOGRX','Japan')

In [8]:
df.head()

,RLP,RM,RX,LOGRLP,LOGRM,LOGRX,dif_LOGRLP,dif_LOGRM,dif_LOGRX
year,,,,,,,,,
1961,49.686283,1.108266e+08,2.683813e+08,3.905729,18.523477,19.407919,0.026884,0.013281,0.009853
1962,53.506872,1.105012e+08,3.213745e+08,3.979810,18.520537,19.588118,0.018967,-0.000159,0.009285
1963,57.561585,1.306210e+08,3.642080e+08,4.052855,18.687811,19.713236,0.018354,0.009032,0.006387
1964,63.503722,1.485290e+08,4.166221e+08,4.151099,18.816291,19.847690,0.024240,0.006875,0.006821
1965,66.112662,1.598079e+08,4.861703e+08,4.191360,18.889483,20.002070,0.009699,0.003890,0.007778


In [ ]:
##http://datatopics.worldbank.org/world-development-indicators/themes/economy.html

In [11]:
pd.options.display.float_format = '{:.3f}'.format

In [12]:
pd.concat([LOGRLP,LOGRM,LOGRX,dif_LOGRLP,dif_LOGRM,dif_LOGRX]).set_index(['country','value','index']).stack().unstack(level=[1,2])

value             LOGRLP          LOGRM          LOGRX       dif_LOGRLP  \
index           constant trend constant trend constant trend   constant   
country                                                                   
Japan   coef    -146.755 0.077 -189.209 0.106 -130.795 0.077      0.691   
        p-value    0.000 0.000    0.000 0.000    0.000 0.000      0.249   

value                  dif_LOGRM        dif_LOGRX         
index            trend  constant  trend  constant  trend  
country                                                   
Japan   coef    -0.000     0.148 -0.000     0.281 -0.000  
        p-value  0.260     0.493  0.510     0.006  0.007

In [25]:
##Unit Root Test
from statsmodels.tsa.stattools import adfuller, kpss

In [105]:
def check_for_stationarity(X, reg,value):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    adf = adfuller(X, regression=reg, maxlag=1)
    result = pd.DataFrame({'ADF':adf[0], 'p-value':adf[1],'lag':adf[2],'Num of Data':adf[3],'1%':round(adf[4]['1%'],3)},index=[value])
    return result

In [106]:
a = check_for_stationarity(df.LOGRLP,'ct','LOGRLP')
b = check_for_stationarity(df.LOGRM,'ct', 'LOGRM')
c = check_for_stationarity(df.LOGRX,'ct','LOGRX')
d = check_for_stationarity(df.dif_LOGRLP,'nc', 'dif_LOGRLP')
e = check_for_stationarity(df.dif_LOGRM,'nc','dif_LOGRM')
f = check_for_stationarity(df.dif_LOGRX,'c','dif_LOGRX')

In [107]:
pd.concat([a,b,c,d,e,f]).T

,LOGRLP,LOGRM,LOGRX,dif_LOGRLP,dif_LOGRM,dif_LOGRX
ADF,-3.707,-2.757,-2.891,-3.407,-3.508,-3.577
p-value,0.022,0.213,0.165,0.001,0.000,0.006
lag,1.000,1.000,1.000,0.000,0.000,0.000
Num of Data,34.000,34.000,34.000,35.000,35.000,35.000
1%,-4.253,-4.253,-4.253,-2.633,-2.633,-3.633
